In [9]:
from sympy import *
import numpy as np
from itertools import product, combinations_with_replacement, chain
from operator import itemgetter, add
from math import factorial
from multiprocessing import Pool, cpu_count
import re
import json
import math
from sympy.physics.units.quantities import Quantity
k = Quantity('k')
x,c = symbols('x c', real=True)

In [20]:

f = sympify('sin(k*k)/x*k', locals={'k': k, 'x': x})
ln(x) == log(x)
# 2*(1026!/(1026-2)!)+((1026+1)!/(2*(1026-1)!))+(1026!/(2*(1026-2)!))+4*1026

True

In [7]:
4/x-3/x

1/x

In [99]:
f = -c*x**3 - c + x**2 + x
type(integrate(f, x, risch=True))

sympy.core.add.Add

In [2]:
float('nan') > 0

False

In [126]:
def depth(expr):
    print(expr.args, expr.func)
    if isinstance(expr, Atom):
        return 0
    else:
        inc = 1
        arg_types = [type(a) for a in expr.args]
        if expr.func == Mul and -1 in expr.args:
            print('mul -1')
            inc = 0
        elif expr.func == Mul and arg_types == [Integer, Pow] and -1 in expr.args[1].args:
            print('pow -1')
            inc = 0
        return inc + max([depth(arg) for arg in expr.args])
    
f = 2/(1-x)
depth(f)


(2, 1/(1 - x)) <class 'sympy.core.mul.Mul'>
pow -1
() <class 'sympy.core.numbers.Integer'>
(1 - x, -1) <class 'sympy.core.power.Pow'>
(1, -x) <class 'sympy.core.add.Add'>
() <class 'sympy.core.numbers.One'>
(-1, x) <class 'sympy.core.mul.Mul'>
mul -1
() <class 'sympy.core.numbers.NegativeOne'>
() <class 'sympy.core.symbol.Symbol'>
() <class 'sympy.core.numbers.NegativeOne'>


2

In [39]:
f = x*exp(x)/4
# f = (2+x)*(x+3)

class ArgStack:
    def __init__(self):
      self.stack = []

    def __len__(self):
        return len([a for a, skip in self.stack if not skip])
  
    def push(self, n_args, skip):
        self.stack.append([n_args, skip])
  
    def pop(self):
        if not len(self.stack):
            return
        if self.stack[-1][0] <= 1:
            slice_end = 0
            for n, _ in reversed(self.stack):
                if n > 1:
                    break
                else:
                    slice_end -= 1
            self.stack = self.stack[:slice_end]
        if len(self.stack):
            self.stack[-1][0] -= 1

# this function is needed, because sympy always converts subtraction to addition
# i.e. 1-x to 1+(-1)*x which introduces an extra leaf, node and level of depth
# it also always converts division to multiplication
# i.e. 4/(1-x) to 4*(1-x)^-1 which also introduces an extra leaf, node and level of depth
def tree_props(f):
    leaves = 0
    nodes = 0
    arg_stack = ArgStack()
    max_depth = 0
    marked_pows = []
    for expr in preorder_traversal(f):
        skip_depth = False
        if expr.func == Mul:
            arg_types = set([type(a) for a in expr.args])
            # multiplication with -1
            if -1 in expr.args:
                leaves -= 1
                nodes -= 2 # nodes are always decreased by two, because *(-1) has two nodes
                skip_depth = True
            # power of -1 which replaces division (can have multiple in one multiplication)
            if Pow in arg_types:
                # have to mark these pows for later, because you can't retrieve parents of args
                pows = [arg for arg in expr.args if type(arg) == Pow and -1 in arg.args]
                marked_pows += pows
                # a multiplication of -1 and a marked pow in the same expression would reduce leaves and depth too much
                if len(pows) and -1 in expr.args:
                    leaves += 1
                    nodes += 2
                    skip_depth = False

        if expr.func == Pow and expr in marked_pows:
            leaves -= 1
            nodes -= 2
            skip_depth = True
            marked_pows.remove(expr)
        
        if isinstance(expr, Atom):
            leaves += 1
            arg_stack.pop()
        else:
            arg_stack.push(len(expr.args), skip_depth)
        nodes += max(1, len(expr.args) - 1)
        max_depth = max(max_depth, len(arg_stack))
    return (max_depth, leaves, nodes)

In [40]:
assert tree_props(x**2) == (1,2,3)
assert tree_props(1/x) == (1,2,3)
assert tree_props(2/x) == (1,2,3)
assert tree_props(x+1) == (1,2,3)
assert tree_props(x-1) == (1,2,3)
assert tree_props(x-2) == (1,2,3)
assert tree_props(1-x) == (1,2,3)
assert tree_props(2-x) == (1,2,3)
assert tree_props(x/3) == (1,2,3)
assert tree_props((2+x)*(x+3)) == (2,4,7)
assert tree_props(x/(2*x + 8)) == (3,4,7)
assert tree_props(x**2/(x - 2)) == (2,4,7)
assert tree_props(exp(sin(x))) == (2,1,3)
assert tree_props(exp(sin(x))+exp(cos(x))) == (3,2,7)
assert tree_props(-2-x + acos(x)) == (2,3,6)
assert tree_props(-x**2 + log(x)) == (2,3,6)
assert tree_props(4*x/(x + 4)) == (2,4,7)
assert tree_props(2*x-1/(x+2)) == (3,5,9)
assert tree_props(1/(x-3)-1/(x+2)) == (3,6,11)
assert tree_props(4/(x-3)-5/(x+2)) == (3,6,11)
assert tree_props(x*exp(x)/4) == (2,3,6) # multiplication with 3 operands
assert tree_props(sqrt(1 - x**2)) == (3,4,7) # sqrt = ^1/2, so it counts as a leaf & node too
assert tree_props(1/(8-x)+exp(7/(sin(x)+5))) == (5,6,13)

In [36]:
with open('uniques_ext_depth2.csv', 'r') as readfile:
    with open('tagged0.csv', 'w') as writefile:
        writefile.write('fn,depth\n')
        for line in readfile:
            writefile.write(line.strip() + ',')
            writefile.write(str(depth(sympify(line.strip()))) + '\n')

In [101]:
# Calculation method taken from https://en.wikipedia.org/wiki/Asymptote#Elementary_methods_for_identifying_asymptotes
# Also mentioned in https://encyclopediaofmath.org/index.php?title=Asymptote
def asymptotes(f):
    a = set()
    for lim in [oo, -oo]:
        m = limit(f/x, x, lim)
        if m.is_real:
            n = limit(f-m*x, x, lim)
            if n.is_real:
                a.add(m*x+n)
    return a

In [102]:
f = (x + 1) / (1 + exp(-x))
asymptotes(f)

{0, x + 1}

In [103]:
s = singularities(1/x, x)
extrema = [(1, 'max')]
f = FiniteSet(sqrt(6), -sqrt(6))
l = list(s.union(f).union(FiniteSet(oo)))
l.sort()
l

[-sqrt(6), 0, sqrt(6), oo]

In [104]:
parse_expr('Union(Interval.open(-oo, 0), Interval.open(0, oo))')

Union(Interval.open(-oo, 0), Interval.open(0, oo))

In [19]:
from sympy.integrals.manualintegrate import manualintegrate, integral_steps
f = 2*x
# calculus.util.continuous_domain(f, x, S.Reals)
steps = integral_steps(f, x)
rules = set()
for match in re.finditer('\w+Rule', str(steps)):
    rules.add(match.group())
print(rules)

{'ConstantTimesRule', 'PowerRule'}


In [7]:
f = sympify('log(x)', locals={'x': x})
singularities(f, x)

{0}

In [58]:
f = log(x)
f.is_rational_function()

In [10]:
def test(t=5):
    print(t)
s = None
test(t=s or 10)

10


In [8]:
i = Symbol('i')
f = x*asin(x)
solutions = solve(f - i, x)
total = EmptySet
for sol in solutions:
    inv = sol.subs(i, x)
    r = calculus.util.function_range(inv, x, S.Reals)
    total += r
total


NotImplementedError: multiple generators [x, asin(x)]
No algorithms are implemented to solve equation -i + x*asin(x)

In [29]:
from sympy.integrals.risch import NonElementaryIntegral

f = 1/log(x)
i = integrate(f, x, risch=True)
len(i.atoms(NonElementaryIntegral))

1

In [7]:
lines = [
    '2*c + x',
    'c + x**2',
    'c + cos(x)',
    '2*c - x',
    'c + exp(x)',
    'c + x**2 + x',
    'c + c/x + x',
    'c + 2*x',
    'c + x + log(x)',
    'c + 2*x',
    '2*c + x',
]
re_mul_int = re.compile('(?<!\*)\*?\d\*(x|c)')

def replace_mul_int(match):
    if match.group(1) == 'x':
        return 'c*x'
    return 'c'

for line in lines:
    print(line)
    line = re.sub(
        re_mul_int,
        replace_mul_int,
        line
    )
    print(line)

2*c + x
c + x
c + x**2
c + x**2
c + cos(x)
c + cos(x)
2*c - x
c - x
c + exp(x)
c + exp(x)
c + x**2 + x
c + x**2 + x
c + c/x + x
c + c/x + x
c + 2*x
c + c*x
c + x + log(x)
c + x + log(x)
c + 2*x
c + c*x
2*c + x
c + x
